# Titanic: Logistic Regression Example
#### http://www.data-mania.com/blog/logistic-regression-example-in-python/

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')

## Logistic regression on the Titanic dataset
* We will put this data into a Pandas DataFrame, called __`titanic`__, and name each of the columns

In [ ]:
titanic = pd.read_csv('https://raw.githubusercontent.com/davewadestein/Gap-Python-2025/refs/heads/main/Data/titanic.csv')
titanic.columns = 'PassengerId Survived Pclass Name Sex Age SibSp Parch Ticket Fare Cabin Embarked'.split()
titanic.head(25)

### VARIABLE DESCRIPTIONS

Survived - Survival (0 = No; 1 = Yes)<br>
Pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)<br>
Name - Name<br>
Sex - Sex<br>
Age - Age<br>
SibSp - Number of Siblings/Spouses Aboard<br>
Parch - Number of Parents/Children Aboard<br>
Ticket - Ticket Number<br>
Fare - Passenger Fare (British pound)<br>
Cabin - Cabin<br>
Embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

### Checking that your target variable is binary
Since we are building a model to predict survival of passengers from the Titanic, our target is going to be "Survived" variable from the titanic dataframe. To make sure that it's a binary variable, let's use Seaborn's __`countplot()`__ function.

In [ ]:
sb.countplot(x='Survived',data=titanic, palette='hls');

Ok, so we the __Survived__ variable is binary (0 - did not survive / 1 - survived)

### Checking for missing values
It's easy to check for missing values by calling the __`isnull()`__ method, and the __`sum()`__ method off of that, to return a tally of all the True values that are returned by the __`isnull()`__ method.

In [ ]:
titanic.isnull().sum()

## How many records are there in the DataFrame?

In [ ]:
titanic.info()

* Ok, so there are only 891 rows in the titanic data frame
* __Cabin__ is almost all missing values, so we can drop that variable completely
* What about age? Age seems like a relevant predictor for survival, right? We'd want to keep the variables, but it has 177 missing values. Yikes!! We are going to need to find a way to approximate for those missing values!

## Taking care of missing values
### Dropping missing values
So let's just go ahead and drop all the variables that aren't relevant for predicting survival. We should at least keep the following:
- __Survived__ - This variable is obviously relevant
- __Pclass__ - Does a passenger's class on the boat affect their survivability?
- __Sex__ - Could a passenger's gender impact their survival rate?
- __Age__ - Does a person's age impact their survival rate?
- __SibSp__ - Does the number of relatives on the boat (that are siblings or a spouse) affect a person survivability? Probably
- __Parch__ - Does the number of relatives on the boat (that are children or parents) affect a person survivability? Probably
- __Fare__ - Does the fare a person paid affect his survivability? Maybe - let's keep it.
- __Embarked__ - Does a person's point of embarkation matter? It depends on how the boat was filled... Let's keep it.

* What about a person's name, ticket number, and passenger ID number? They're irrelavant for predicting survivability. And as you recall, the cabin variable is almost all missing values, so we can just drop all of these.

In [ ]:
titanic_data = titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
titanic_data.head()

* Now we have the DataFrame reduced down to only relevant variables, but now we need to deal with the missing values in the age variable.

### Imputing missing values
Let's look at how passenger age is related to their class as a passenger on the ship

In [ ]:
sb.boxplot(x='Pclass', y='Age', data=titanic_data, palette='hls');

In [ ]:
titanic_data.head()

Speaking roughly, we could say that the younger a passenger is, the more likely it is for them to be in 3rd class. The older a passenger is, the more likely it is for them to be in 1st class. So there is a loose relationship between these variables. So, let's write a function that approximates a passengers age, based on their class. From the box plot, it looks like the average age of 1st class passengers is about 37, 2nd class passengers is 29, and 3rd class pasengers is 24.

So let's write a function that finds each null value in the Age variable, and for each null, checks the value of the Pclass and assigns an age value according to the average age of passengers in that class.

In [ ]:
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    class_to_age = { 1: 37, 2: 29, 3: 24 }

    if pd.isnull(Age):
        return class_to_age[Pclass]
    else:
        return Age

When we apply the function and check again for null values, we see that there are no more null values in the age variable.

In [ ]:
titanic_data['Age'] = titanic_data[['Age', 'Pclass']].apply(age_approx, axis=1)
titanic_data.isnull().sum()

* There are 2 null values in the embarked variable. We can drop those 2 records without losing too much important information from our dataset, so we will do that.

In [ ]:
titanic_data.dropna(inplace=True)
titanic_data.isnull().sum()

### Converting categorical variables to a dummy variables
The next thing we need to do is reformat our variables so that they work with the model. Specifically, we need to reformat the __Sex__ and __Embarked__ variables into numeric variables.

In [ ]:
gender = pd.get_dummies(titanic_data['Sex'], drop_first=True)
gender.head()

In [ ]:
embark_location = pd.get_dummies(titanic_data['Embarked'], drop_first=True)
embark_location.head(25)

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.drop(['Sex', 'Embarked'], axis=1, inplace=True)
titanic_data.head()

In [ ]:
titanic_dmy = pd.concat([titanic_data, gender, embark_location], axis=1)
titanic_dmy.head()

Now we have a dataset with all the variables in the correct format!

### Checking for independence between features

In [ ]:
sb.heatmap(titanic_dmy.corr(), cmap='coolwarm');

### "__Fare__ and __Pclass__ are not independent of each other, so I am going to drop these."

(The above are the exact words of Lillian Pierson, yet they are confusing. We'd expect to keep one of them and not the other. A later comment by her suggests the she was concerned about multicollinearity, i.e., two correlated variables. A typical remedy for this is in fact to drop one of the highly-correlated variables, but we can see that different data scientists make different choices.)

In [ ]:
titanic_dmy.drop(['Fare', 'Pclass'], axis=1, inplace=True)
titanic_dmy.head()

### Checking that your dataset size is sufficient
We have 6 predictive features that remain. The rule of thumb is 50 records per feature...so we need to have at least 300 records in this dataset. Let's check again.

In [ ]:
titanic_dmy.info()

Ok, we have 889 records so we are fine.

In [ ]:
X = titanic_dmy.iloc[:,1:].values
y = titanic_dmy.iloc[:,0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

### Deploying and evaluating the model

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

In [ ]:
y_pred = LogReg.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

* Remember that correct predictions are on the diagonal

In [ ]:
print(classification_report(y_test, y_pred, target_names=['died', 'survived']))

In [ ]:
# F1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
137 / (137 + 27) # TP / (TP + FN)

In [ ]:
137 / (137 + 34) # TP / (TP + FP)